In [25]:
#case2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv('data/HR_data_2.csv')
#print header
print(data.head())


   Unnamed: 0  HR_TD_Mean  HR_TD_Median  HR_TD_std  HR_TD_Min  HR_TD_Max  \
0           0   78.663878         76.70   7.480043      67.25      92.48   
1           1   76.540732         76.61   2.584756      69.82      82.33   
2           2   78.173563         77.92   2.681255      72.22      82.80   
3           3   83.073688         83.88   7.363598      69.42      96.12   
4           4   72.281250         72.91   3.193762      64.95      79.98   

   HR_TD_AUC  HR_TD_Kurtosis  HR_TD_Skew  HR_TD_Slope_min  ...  upset  \
0  23048.450       -1.091448    0.369955          -0.7300  ...    1.0   
1  23959.920       -0.245338    0.338732          -0.3600  ...    2.0   
2  20324.605       -0.615922   -0.233047          -0.6300  ...    1.0   
3  24924.300       -0.866610   -0.046021          -0.4650  ...    1.0   
4  23052.100        0.200401   -0.560948          -0.3725  ...    3.0   

   hostile  alert  ashamed  inspired  nervous  attentive  afraid  active  \
0      1.0    2.0      1.0  

In [26]:
import torch

In [27]:
data.columns

Index(['Unnamed: 0', 'HR_TD_Mean', 'HR_TD_Median', 'HR_TD_std', 'HR_TD_Min',
       'HR_TD_Max', 'HR_TD_AUC', 'HR_TD_Kurtosis', 'HR_TD_Skew',
       'HR_TD_Slope_min', 'HR_TD_Slope_max', 'HR_TD_Slope_mean', 'HR_TD_Slope',
       'TEMP_TD_Mean', 'TEMP_TD_Median', 'TEMP_TD_std', 'TEMP_TD_Min',
       'TEMP_TD_Max', 'TEMP_TD_AUC', 'TEMP_TD_Kurtosis', 'TEMP_TD_Skew',
       'TEMP_TD_Slope_min', 'TEMP_TD_Slope_max', 'TEMP_TD_Slope_mean',
       'TEMP_TD_Slope', 'EDA_TD_P_Mean', 'EDA_TD_P_Median', 'EDA_TD_P_std',
       'EDA_TD_P_Min', 'EDA_TD_P_Max', 'EDA_TD_P_AUC', 'EDA_TD_P_Kurtosis',
       'EDA_TD_P_Skew', 'EDA_TD_P_Slope_min', 'EDA_TD_P_Slope_max',
       'EDA_TD_P_Slope_mean', 'EDA_TD_P_Slope', 'EDA_TD_T_Mean',
       'EDA_TD_T_Median', 'EDA_TD_T_std', 'EDA_TD_T_Min', 'EDA_TD_T_Max',
       'EDA_TD_T_AUC', 'EDA_TD_T_Kurtosis', 'EDA_TD_T_Skew',
       'EDA_TD_T_Slope_min', 'EDA_TD_T_Slope_max', 'EDA_TD_T_Slope_mean',
       'EDA_TD_T_Slope', 'EDA_TD_P_Peaks', 'EDA_TD_P_RT', 'EDA_TD_P_R

In [28]:
dir_path = data['raw_data_path'][0]

In [29]:
hr = pd.read_csv('data/' + dir_path + '/HR.csv', header=0, index_col=0, parse_dates=True)
hr


,HR,time
0,71.93,2022-10-26 16:41:36
1,72.23,2022-10-26 16:41:37
2,72.48,2022-10-26 16:41:38
3,72.53,2022-10-26 16:41:39
4,72.60,2022-10-26 16:41:40
...,...,...
289,86.28,2022-10-26 16:46:25
290,86.18,2022-10-26 16:46:26
291,85.97,2022-10-26 16:46:27
292,85.82,2022-10-26 16:46:28


In [ ]:
import gc
import pandas as pd
from tqdm import tqdm
from etna.transforms.embeddings.models import TS2VecEmbeddingModel

# Load pretrained model
model = TS2VecEmbeddingModel.load(model_name='ts2vec_tiny')
model.embedding_model.batch_size = 1

def get_embeddings(dir_path, model, signal):
    file_info = {
        "HR": "HR.csv",
        "EDA": "EDA.csv",
        "BVP": "BVP.csv",
        "TEMP": "TEMP.csv"
    }

    processed_dfs = {}

    for signal_name, filename in file_info.items():
        df = pd.read_csv(f'data/{dir_path}/{filename}', parse_dates=["time"])
        df["t_seconds"] = (df["time"] - df["time"].iloc[0]).dt.total_seconds()
        df = df.drop(columns=["time"])
        cols = [col for col in df.columns if col != "t_seconds"]
        df = df[["t_seconds"] + cols]
        processed_dfs[signal_name] = df.drop(columns=['Unnamed: 0'], errors='ignore')

    merged = pd.merge(processed_dfs['HR'], processed_dfs['EDA'], on='t_seconds', how='outer')
    merged = pd.merge(merged, processed_dfs['BVP'], on='t_seconds', how='outer')
    merged = pd.merge(merged, processed_dfs['TEMP'], on='t_seconds', how='outer')
    merged = merged.interpolate(method='linear', limit_direction='both')

    dat_1 = merged[signal].to_numpy()
    dat_1 = dat_1[None, :, None]
    return model.embedding_model.encode(dat_1)

def get_all_embeddings(data, model):
    signals = ["HR", "EDA", "BVP", "TEMP"]
    for signal in signals:
        embeddings_col = f"{signal}_embeddings"
        data[embeddings_col] = None

    for i in tqdm(range(len(data))):
        dir_path = data.loc[i, 'raw_data_path']
        for signal in ["HR", "EDA", "BVP", "TEMP"]:
            embeddings = get_embeddings(dir_path, model, signal)
            data.at[i, f"{signal}_embeddings"] = np.mean(embeddings[0], axis=0).tolist()
        

    return data

# Process and save all embeddings to a single CSV
data = get_all_embeddings(data, model)
data.to_csv("data_with_all_embeddings.csv", index=False)


/home/khalil/miniconda3/envs/env_works/lib/python3.10/site-packages/etna/transforms/embeddings/models/ts2vec.py:311: UserWarning: Path /home/khalil/.etna/embeddings/ts2vec/ts2vec_tiny.zip already exists. Model ts2vec_tiny will not be downloaded. Loading existing local model.
  warnings.warn(
  0%|          | 0/312 [00:00<?, ?it/s]

100%|██████████| 312/312 [02:14<00:00,  2.31it/s]


In [27]:
##save data as csv under name 'HR_data_2_embeddings.csv' 
data.to_csv('data/HR_data_2_embeddings.csv', index=False)

#use ast literal_eval to convert string to list
#import ast
#data['HR_embeddings'] = data['HR_embeddings'].apply(ast.literal_eval)
#data['EDA_embeddings'] = data['EDA_embeddings'].apply(ast.literal_eval)
#data['BVP_embeddings'] = data['BVP_embeddings'].apply(ast.literal_eval)
#data['TEMP_embeddings'] = data['TEMP_embeddings'].apply(ast.literal_eval)

In [45]:
#convert to numpy array
hr_embeddings = np.array(data['HR_embeddings'].tolist())
eda_embeddings = np.array(data['EDA_embeddings'].tolist())
bvp_embeddings = np.array(data['BVP_embeddings'].tolist())
temp_embeddings = np.array(data['TEMP_embeddings'].tolist())
#stack all embeddings
all_embeddings = np.hstack((hr_embeddings, eda_embeddings, bvp_embeddings, temp_embeddings))
all_embeddings.shape

(312, 64)

In [46]:
y = data['Frustrated']

In [48]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(all_embeddings, y, test_size=0.2, random_state=42)

# Fit regression model
reg = LinearRegression()
reg.fit(X_train, y_train)

# Predict and evaluate
y_pred = reg.predict(X_test)
print("MSE:", mean_absolute_error(y_test, y_pred))


MSE: 2.1218350704414126


In [42]:
data

,Unnamed: 0,HR_TD_Mean,HR_TD_Median,HR_TD_std,HR_TD_Min,HR_TD_Max,HR_TD_AUC,HR_TD_Kurtosis,HR_TD_Skew,HR_TD_Slope_min,...,hostile,alert,ashamed,inspired,nervous,attentive,afraid,active,determined,embeddings
0,0,78.663878,76.700,7.480043,67.25,92.48,23048.450,-1.091448,0.369955,-0.730000,...,1.0,2.0,1.0,2.0,2.0,3.0,1.0,2.0,2.0,"[[[-0.05565757, -0.03793873, -0.04383092, -0.0..."
1,1,76.540732,76.610,2.584756,69.82,82.33,23959.920,-0.245338,0.338732,-0.360000,...,1.0,3.0,2.0,2.0,2.0,3.0,1.0,3.0,3.0,"[[[-0.05565757, -0.03793873, -0.04383092, -0.0..."
2,2,78.173563,77.920,2.681255,72.22,82.80,20324.605,-0.615922,-0.233047,-0.630000,...,1.0,2.0,1.0,3.0,2.0,3.0,2.0,3.0,3.0,"[[[-0.05565757, -0.03793873, -0.04383092, -0.0..."
3,3,83.073688,83.880,7.363598,69.42,96.12,24924.300,-0.866610,-0.046021,-0.465000,...,1.0,2.0,1.0,3.0,2.0,3.0,2.0,3.0,3.0,"[[[-0.05565757, -0.03793873, -0.04383092, -0.0..."
4,4,72.281250,72.910,3.193762,64.95,79.98,23052.100,0.200401,-0.560948,-0.372500,...,1.0,3.0,2.0,3.0,3.0,4.0,2.0,4.0,4.0,"[[[-0.05565757, -0.03793873, -0.04383092, -0.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,307,82.019065,78.830,8.794206,69.73,97.18,26244.705,-1.386324,0.251382,-0.517500,...,2.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,"[[[-0.05565757, -0.03793873, -0.04383092, -0.0..."
308,308,103.274248,104.625,14.208117,68.13,125.52,31523.695,-0.338091,-0.521254,-0.962500,...,1.0,3.0,1.0,1.0,1.0,4.0,1.0,3.0,4.0,"[[[-0.05565757, -0.03793873, -0.04383092, -0.0..."
309,309,99.065000,97.490,10.079197,86.55,119.75,28235.230,-0.740136,0.629730,-0.657500,...,1.0,3.0,1.0,3.0,1.0,3.0,1.0,3.0,4.0,"[[[-0.05565757, -0.03793873, -0.04383092, -0.0..."
310,310,91.793354,97.180,19.234169,60.90,120.67,28909.835,-1.229115,-0.181431,-0.717500,...,1.0,3.0,1.0,3.0,1.0,4.0,1.0,4.0,4.0,"[[[-0.05565757, -0.03793873, -0.04383092, -0.0..."


In [8]:
dat_1 = hr_eda_bvp_temp.to_numpy()  # shape: (T, 4)
dat_1 = dat_1[np.newaxis, :, :]

In [32]:
dat_1.shape

(1, 18855, 5)

In [9]:
dat_1.shape

(1, 18855, 5)

In [10]:
from etna.transforms.embeddings.models import TS2VecEmbeddingModel

/home/khalil/miniconda3/envs/env_works/lib/python3.10/site-packages/etna/settings.py:51: UserWarning: etna[chronos] is not available, to install it, run `pip install etna[chronos]`
  warnings.warn("etna[chronos] is not available, to install it, run `pip install etna[chronos]`")
/home/khalil/miniconda3/envs/env_works/lib/python3.10/site-packages/etna/settings.py:66: UserWarning: etna[timesfm] is not available, to install it, run `pip install etna[timesfm]`
  warnings.warn("etna[timesfm] is not available, to install it, run `pip install etna[timesfm]`")
/home/khalil/miniconda3/envs/env_works/lib/python3.10/site-packages/etna/settings.py:82: UserWarning: etna[prophet] is not available, to install it, run `pip install etna[prophet]`
  warnings.warn("etna[prophet] is not available, to install it, run `pip install etna[prophet]`")
/home/khalil/miniconda3/envs/env_works/lib/python3.10/site-packages/etna/settings.py:91: UserWarning: etna[classification] is not available, to install it, run `pi

In [ ]:
from etna.transforms.embeddings.models import TS2VecEmbeddingModel

# List available pretrained models
available_models = TS2VecEmbeddingModel.list_models()
print(available_models)

# Load a specific pretrained model
model = TS2VecEmbeddingModel.load(model_name='ts2vec_tiny')
model.embedding_model.batch_size = 1


['ts2vec_tiny']


In [20]:
dat_1.shape

(1, 18855, 5)

In [21]:
model.embedding_model.batch_size = 1

In [29]:
dat_1[:,:,0,None].shape

(1, 18855, 1)

In [31]:
model.embedding_model.encode(dat_1[:,:,0,None]).shape

(1, 18855, 16)

In [13]:
df.iloc[0]

t_seconds      0.00
Unnamed: 0     0.00
TEMP          31.17
Name: 0, dtype: float64

In [25]:
#Use the TS2Vec model to encode the time series data, pooling the time series data to a single vector
from ts2vec import TS2Vec
model = TS2Vec(input_dims=5)
embedding = model.encode(dat_1,encoding_window='full_sequence')
embedding.shape

(1, 18855, 320)

In [27]:
#maxpooling across dim=1 using numpy
embedding = np.max(embedding, axis=1)
embedding.shape


(1, 320)

In [20]:
embedding[0][0]

array([ -9.485781  ,  13.332074  , -16.295546  ,  -5.3082547 ,
        -8.434439  ,  -7.246311  ,  -0.92227983,  10.377557  ,
         5.8486757 ,  -4.2260165 ,  10.616538  ,  -3.049604  ,
        -8.237917  ,   9.703344  ,   8.291063  , -13.267677  ,
         2.1475034 ,  -2.1767588 ,  -9.245678  ,  -1.6883235 ,
         7.218567  ,  -5.460306  ,   0.48868537,  -8.35013   ,
         0.81740713,  18.342293  ,  12.174667  ,  -0.27726412,
        14.329578  , -10.779439  ,  -5.621278  ,  -3.6623874 ,
         7.658205  , -13.077543  ,   0.2083444 ,   6.2842484 ,
        23.968235  ,  -3.4631057 ,   5.0482955 ,   2.4785128 ,
       -11.997673  ,   1.6916022 ,  24.21688   , -22.794752  ,
         0.86318827,  -0.5043845 ,  38.091522  , -20.266485  ,
        -2.5980997 ,  -8.165831  ,   1.6012913 ,   4.8163714 ,
         2.940488  ,   6.66364   , -14.669049  ,  11.283632  ,
       -12.573567  ,   9.048862  ,  -0.9166999 ,  13.437372  ,
        -7.2853518 ,   2.5603375 ,  16.68581   ,   1.63